# PuLPとGurobi/Pythonによる混合整数最適化問題

* 簡単な例題
* 多制約ナップサック問題


## 例題 パズル（整数最適化）

鶴と亀と蛸が何匹かずついる
頭の数を足すと $32$，足の数を足すと $80$ になる．
亀と蛸の数の和を一番小さくするような匹数を求めよ．

\begin{array}{l c c }
  \mbox{minimize}    &  y + z          \\
  \mbox{subject to}  & x + y + z  = 32 \\
                     & 2x +4y +8z = 80 \\
               & x,y,z  \mbox{ は非負の整数} 
\end{array}



In [3]:
#from gurobipy import *
from mypulp import *
model = Model("puzzle")
x = model.addVar(vtype="I", name="x")
y = model.addVar(vtype="I", name="y")
z = model.addVar(vtype="I", name="z")
model.update()

model.addConstr(x + y + z == 32, "Heads")
model.addConstr(2*x + 4*y + 8*z == 80, "Legs")

model.setObjective(y + z, GRB.MINIMIZE)

model.optimize()

print("Opt. Val.=",model.ObjVal)
print("(x,y,z)=",(x.X,y.X,z.X))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mukaishunsuke/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/9hrzy57n5rb428pvqftbcxwr0000gn/T/60fa143d4e51493ca8fd239245e06cf9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/9hrzy57n5rb428pvqftbcxwr0000gn/T/60fa143d4e51493ca8fd239245e06cf9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 22 RHS
At line 25 BOUNDS
At line 29 ENDATA
Problem MODEL has 2 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.66667 - 0.00 seconds
Cgl0003I 0 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 4 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 

## 例題 多制約ナップサック問題（0-1整数最適化）

* $n$ 個のアイテム，$m$本の制約
* 各々のアイテム $j=1,2,\ldots,n$ の価値 $v_j ~(\geq 0)$，
* アイテム $j$ の制約 $i=1,2,\ldots,m$ に対する重み $a_{ij}~(\geq 0)$
* 制約 $i$ に対する制約の上限値 $b_i~(\geq 0)$ 



* アイテムの番号の集合 $I =\{1,2,\ldots,n \}$
* 制約の番号の集合 $J =\{1,2,\ldots,m\}$ 
* アイテム $j$ をナップサックに詰めるとき $1$，
それ以外のとき $0$ になる $0$-$1$ 変数 $x_j$

定式化

\begin{array}{l l l}
   \mbox{maximize}     & \displaystyle\sum_{j \in J} v_j x_j          &  \\
   \mbox{subject to}  &  \displaystyle\sum_{j \in J} a_{ij} x_j \leq b_i & \forall i \in I    \\
                       & x_j \in \{0,1\} &     \forall j \in J
\end{array}

In [4]:
#from gurobipy import *
from mypulp import *

J,v = multidict({1:16, 2:19, 3:23, 4:28})
a = {(1,1):2,    (1,2):3,    (1,3):4,    (1,4):5,
     (2,1):3000, (2,2):3500, (2,3):5100, (2,4):7200,
    }
I,b = multidict({1:7, 2:10000})
    
model = Model('mkp')
x = {}
for j in J:
    x[j] = model.addVar(vtype='B', name='x({0})'.format(j))
model.update()
     
for i in I:
    model.addConstr(quicksum(a[i,j]*x[j] for j in J) <= b[i], 'Capacity({0})'.format(i))

model.setObjective(quicksum(v[j]*x[j] for j in J), GRB.MAXIMIZE)

model.optimize()
print('Optimal value=',model.ObjVal)

EPS = 1.e-6
for v in model.getVars():
    if v.X > EPS:
        print( v.VarName,v.X)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mukaishunsuke/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/09/9hrzy57n5rb428pvqftbcxwr0000gn/T/b6a92834f3b94e6380e0f8c3d61f0ece-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/09/9hrzy57n5rb428pvqftbcxwr0000gn/T/b6a92834f3b94e6380e0f8c3d61f0ece-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 28 RHS
At line 31 BOUNDS
At line 36 ENDATA
Problem MODEL has 2 rows, 4 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 46.5 - 0.00 seconds
Cgl0004I processed model has 2 rows, 4 columns (4 integer (4 of which binary)) and 8 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.5
Cbc0038I Pass   1: suminf.    0.50000 (1) obj. -46.5